In [1]:
import pandas as pd
import numpy as np

import torch
from transformers import AutoModel, AutoTokenizer
from tqdm.auto import tqdm


from MyLib.Utilities import IndoBertModelRep
from sklearn.metrics.pairwise import cosine_similarity


Import libary tercover program bisa jalan....


### Load dataset

In [7]:
baseData = pd.read_csv("./data/final_dataset.csv")
listData = baseData.Isi.tolist()

In [16]:
areError = [a for a in [len(baseData.Isi[a].split(" ")) for a in range(len(baseData))] if a >= 128]

### Modeling

In [3]:
model_benchmark = "indobenchmark/indobert-base-p1"
model = AutoModel.from_pretrained(model_benchmark)
tokinizer = AutoTokenizer.from_pretrained(model_benchmark)

In [4]:
rep = IndoBertModelRep(listData[:5],model,tokinizer)

transforming tokens:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
rep.fit_transform(2)

Training:   0%|          | 0/3 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.761045,-0.709757,0.909030,0.437635,0.279120,-0.928241,0.344509,0.341682,-0.023158,0.867788,...,-0.226692,0.303870,0.074755,0.129440,-0.261591,-0.283898,1.256523,0.985545,-0.044319,-0.004940
1,-0.148623,0.422687,0.631739,-0.052750,0.257958,-0.218959,-0.298142,0.168570,0.049852,1.164452,...,-0.235537,-0.567633,-0.394888,0.432861,-0.044941,-0.346878,0.499489,0.718912,0.376147,0.598184
2,-0.211517,-0.076270,0.920100,0.289120,0.729705,-0.882746,-0.046888,0.207160,-0.491770,1.196646,...,-0.611195,0.275531,-0.152839,-0.757432,-0.596600,0.127545,-0.091879,0.158900,0.068856,0.872876
3,-0.487513,0.480990,0.757479,0.395799,0.095389,-0.532911,-0.860657,0.758361,0.533177,0.052654,...,0.201894,-0.417932,-0.341456,0.097244,0.000644,-0.975596,0.245799,0.530606,-0.505319,0.221170
4,0.125290,0.791539,0.416609,0.781668,-0.055229,-0.217871,-0.494227,0.949064,0.579342,0.005496,...,0.182095,-0.576178,-0.502916,-0.025782,-0.055154,-0.885915,0.861456,0.262853,-0.651098,0.508318


<bound method IndoBertModelRep.__transformSentence of <MyLib.Utilities.IndoBertModelRep object at 0x00000186B10C1690>>

### define the Batching dataset

In [10]:
class BatchingDataset(object):

    def __init__(self, data : list, model = model, tokinizer = tokinizer):
        self.model = model
        self.tokinizer = tokinizer
        self.data = self.__process(data)

    def __process(self,data):
        max_lenght = 128
        tokens = {"input_ids":[],"attention_mask":[]}
        for sentence in tqdm(data,desc="transforming tokens"):
            text = tokinizer.encode_plus(
                sentence,
                max_length = max_lenght,
                padding = "max_length",
                truncation = True,
                return_tensors = "pt"
            )

            tokens["input_ids"].append(text["input_ids"][0])
            tokens["attention_mask"].append(text["attention_mask"][0])

        tokens["input_ids"] = torch.stack(tokens["input_ids"])
        tokens["attention_mask"] = torch.stack(tokens["attention_mask"])

        return tokens
    

    def __fittingData(self,dataToken : dict) -> np.ndarray:
            
            self.model.eval()
            output = self.model(**dataToken)
            embeddings = output.last_hidden_state

            attention = dataToken["attention_mask"]
            attention.unsqueeze(-1).shape

            val = attention.unsqueeze(-1)
            mask = val.expand(embeddings.shape).float()
            mask_embeddings = embeddings * mask

            summed = torch.sum(mask_embeddings,1)
            summed_mask = mask.sum(1)
            clamp_summed_mask = torch.clamp(summed_mask,min=1e-9)
            mean_pooling = summed / clamp_summed_mask

            return mean_pooling.detach().numpy()    


    def fit_transform(self, batch : int,useGPU : bool = False) -> np.ndarray:
        totalBatch = (len(self.data["input_ids"]) // batch) + 1
        baseBatch = 0
        baseEndBatch = batch
        limitBatch = len(self.data["input_ids"])
        self.arrayData = []

        for i in tqdm(range(totalBatch),desc="Training"):
            dataTrain = {"input_ids" : self.data["input_ids"][baseBatch:baseEndBatch if baseEndBatch < limitBatch else limitBatch],
                         "attention_mask":self.data["attention_mask"][baseBatch:baseEndBatch if baseEndBatch < limitBatch else limitBatch]}

            baseBatch += batch
            baseEndBatch += batch
            clean = self.__fittingData(dataTrain)
            self.arrayData.append(pd.DataFrame(clean))

            
        self.bankData = pd.concat(self.arrayData).reset_index(drop=True)
        self.transformSentence = self.__transformSentence
        return self.bankData
    


    def __transformSentence(self,sentence : str, returnPredict : bool = False) -> np.ndarray | list:
        tokens = self.__process([sentence])
        arrayToken = self.__fittingData(tokens)
        

        #kalkulasi kemiripan
        cos = cosine_similarity(arrayToken,self.bankData.values)
        result = dict(index = cos[0],result = np.flip(cos.argsort())[0])

        if(returnPredict):
             return [result, pd.DataFrame(arrayToken)]
        
        return result
        
        



            

In [11]:
batchingDataset = BatchingDataset(listData[:5])

transforming tokens:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
batchingDataset.fit_transform(2)

Training:   0%|          | 0/3 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.761045,-0.709757,0.909030,0.437635,0.279120,-0.928241,0.344509,0.341682,-0.023158,0.867788,...,-0.226692,0.303870,0.074755,0.129440,-0.261591,-0.283898,1.256523,0.985545,-0.044319,-0.004940
1,-0.148623,0.422687,0.631739,-0.052750,0.257958,-0.218959,-0.298142,0.168570,0.049852,1.164452,...,-0.235537,-0.567633,-0.394888,0.432861,-0.044941,-0.346878,0.499489,0.718912,0.376147,0.598184
2,-0.211517,-0.076270,0.920100,0.289120,0.729705,-0.882746,-0.046888,0.207160,-0.491770,1.196646,...,-0.611195,0.275531,-0.152839,-0.757432,-0.596600,0.127545,-0.091879,0.158900,0.068856,0.872876
3,-0.487513,0.480990,0.757479,0.395799,0.095389,-0.532911,-0.860657,0.758361,0.533177,0.052654,...,0.201894,-0.417932,-0.341456,0.097244,0.000644,-0.975596,0.245799,0.530606,-0.505319,0.221170
4,0.125290,0.791539,0.416609,0.781668,-0.055229,-0.217871,-0.494227,0.949064,0.579342,0.005496,...,0.182095,-0.576178,-0.502916,-0.025782,-0.055154,-0.885915,0.861456,0.262853,-0.651098,0.508318


In [96]:
(result,arraySentence) = batchingDataset.transformSentence(listData[0],True)

transforming tokens:   0%|          | 0/1 [00:00<?, ?it/s]

In [97]:
result

{'index': array([1.0000002 , 0.45350268, 0.7043845 , 0.5020958 , 0.48358178],
       dtype=float32),
 'result': array([0, 2, 3, 4, 1], dtype=int64)}

In [98]:
arraySentence

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.761044,-0.709757,0.90903,0.437635,0.279119,-0.928241,0.344509,0.341683,-0.023158,0.867788,...,-0.226692,0.30387,0.074755,0.129439,-0.261591,-0.283898,1.256522,0.985545,-0.044318,-0.00494
